## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [2]:
NAME = "Antonios Keremidis"
AEM = "9717"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [3]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [4]:
# import urllib.request
# url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
# filename_train = 'train_set.csv'
# urllib.request.urlretrieve(url_train, filename_train)
# url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
# filename_test = 'test_set_noclass.csv'
# urllib.request.urlretrieve(url_test, filename_test)

In [5]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. Otherwise, you can use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three **simple** estimators/classifiers. Test both soft and hard voting and choose the best one. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   Probabilistic Models (Naive Bayes)
*   KNN Models  

In [6]:
# BEGIN CODE HERE
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import KFold, cross_validate

cls1 = LogisticRegression(random_state=RANDOM_STATE, max_iter=200,
                          solver="lbfgs", multi_class="multinomial")  # Classifier #1
cls2 = KNeighborsClassifier(n_neighbors=8)  # Classifier #2
cls3 = DecisionTreeClassifier(random_state=RANDOM_STATE)  # Classifier #3

soft_vcls = VotingClassifier(estimators=[('lr', cls1), ('kn', cls2), ('dt', cls3)],
                             voting='soft', n_jobs=-1)  # Soft Voting Classifier
hard_vcls = VotingClassifier(estimators=[('lr', cls1), ('kn', cls2), ('dt', cls3)],
                             voting='hard', n_jobs=-1)  # Hard Voting Classifier

scoring = {'accuracy': make_scorer(accuracy_score),
           'f1_score': make_scorer(f1_score)}

kfold = KFold(n_splits=7, shuffle=True, random_state=RANDOM_STATE)

svlcs_scores = cross_validate(
    soft_vcls, X, y, cv=kfold, scoring=scoring, n_jobs=-1)

s_avg_fmeasure = np.mean(svlcs_scores['test_f1_score'])
s_avg_accuracy = np.mean(svlcs_scores['test_accuracy'])

hvlcs_scores = cross_validate(
    hard_vcls, X, y, cv=kfold, scoring=scoring, n_jobs=-1)
    
h_avg_fmeasure = np.mean(hvlcs_scores['test_f1_score'])
h_avg_accuracy = np.mean(hvlcs_scores['test_accuracy'])
# END CODE HERE


/opt/homebrew/Caskroom/miniforge/base/envs/sandbox/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/envs/sandbox/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

In [7]:
print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=200,
                                                 multi_class='multinomial',
                                                 random_state=42)),
                             ('kn', KNeighborsClassifier(n_neighbors=8)),
                             ('dt', DecisionTreeClassifier(random_state=42))],
                 n_jobs=-1, voting='soft')
F1 Weighted-Score: 0.8506 & Balanced Accuracy: 0.8215


You should achive above 82% (Soft Voting Classifier)

In [8]:
print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=200,
                                                 multi_class='multinomial',
                                                 random_state=42)),
                             ('kn', KNeighborsClassifier(n_neighbors=8)),
                             ('dt', DecisionTreeClassifier(random_state=42))],
                 n_jobs=-1)
F1 Weighted-Score: 0.8529 & Balanced Accuracy: 0.8272


You should achieve above 80% in both! (Hard Voting Classifier)

### 1.2 Stacking ###
Create a stacking classifier which uses two more complex estimators. Try different simple classifiers (like the ones mentioned before) for the combination of the initial estimators. Report your results in the following cell.

Consider as complex estimators the following:

*   Random Forest
*   SVM
*   Gradient Boosting
*   MLP




In [9]:
# BEGIN CODE HERE
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

cls1 = MLPClassifier(activation="relu", max_iter=2000,
                     random_state=RANDOM_STATE)
cls2 = GradientBoostingClassifier(
    random_state=RANDOM_STATE, n_estimators=20, learning_rate=0.05, max_features=2, max_depth=2)
cls3 = RandomForestClassifier(
    criterion="gini", max_depth=8, n_jobs=-1, random_state=RANDOM_STATE)

kfold = KFold(n_splits=6, shuffle=True, random_state=RANDOM_STATE)

scls = StackingClassifier(estimators=[('mlp', cls1), ('gbc', cls2), (
    'rfc', cls3)], stack_method='auto', n_jobs=-1, passthrough=False)

scores = {'accuracy': make_scorer(
    accuracy_score), 'f1_score': make_scorer(f1_score)}

results = cross_validate(scls, X, y, cv=kfold, n_jobs=-1, scoring=scores)

avg_fmeasure = np.mean(results['test_f1_score'])
avg_accuracy = np.mean(results['test_accuracy'])
# END CODE HERE


In [10]:
print("Classifier:")
print(scls)
print("F1 Weighted Score: {} & Balanced Accuracy: {}".format(round(avg_fmeasure,4), round(avg_accuracy,4)))

Classifier:
StackingClassifier(estimators=[('mlp',
                                MLPClassifier(max_iter=2000, random_state=42)),
                               ('gbc',
                                GradientBoostingClassifier(learning_rate=0.05,
                                                           max_depth=2,
                                                           max_features=2,
                                                           n_estimators=20,
                                                           random_state=42)),
                               ('rfc',
                                RandomForestClassifier(max_depth=8, n_jobs=-1,
                                                       random_state=42))],
                   n_jobs=-1)
F1 Weighted Score: 0.8811 & Balanced Accuracy: 0.858


You should achieve above 85% in both

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements.  

In [11]:
# BEGIN CODE HERE
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier

ens1 = AdaBoostClassifier(random_state=RANDOM_STATE)
ens2 = RandomForestClassifier(n_jobs=-1, random_state=RANDOM_STATE)
ens3 = BaggingClassifier(n_jobs=-1, random_state=RANDOM_STATE)
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)

f_measures = dict()
accuracies = dict()

scores = {'accuracy': make_scorer(
    accuracy_score), 'f1_score': make_scorer(f1_score)}

kfold = KFold(n_splits=7, random_state=RANDOM_STATE, shuffle=True)

results = cross_validate(tree, X, y, cv=kfold, scoring=scores, n_jobs=-1)

f_measures["dt"] = results['test_f1_score']
accuracies["dt"] = results['test_accuracy']

results = cross_validate(ens1, X, y, cv=kfold, scoring=scores, n_jobs=-1)

f_measures["rf"] = results['test_f1_score']
accuracies["rf"] = results['test_accuracy']

results = cross_validate(ens2, X, y, cv=kfold, scoring=scores, n_jobs=-1)

f_measures["bag"] = results['test_f1_score']
accuracies["bag"] = results['test_accuracy']

results = cross_validate(ens3, X, y, cv=kfold, scoring=scores, n_jobs=-1)

f_measures["ada"] = results['test_f1_score']
accuracies["ada"] = results['test_accuracy']
# END CODE HERE

In [12]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1 Weighted:{}".format(name,np.round(score,4)))
for name,score in accuracies.items():
    print("Classifier:{} -  BalancedAccuracy:{}".format(name,np.round(score,4)))

AdaBoostClassifier(random_state=42)
RandomForestClassifier(n_jobs=-1, random_state=42)
BaggingClassifier(n_jobs=-1, random_state=42)
DecisionTreeClassifier(random_state=42)
Classifier:dt -  F1 Weighted:[0.7767 0.7591 0.7515 0.7115 0.7432 0.7947 0.7557]
Classifier:rf -  F1 Weighted:[0.8199 0.8621 0.8129 0.7735 0.825  0.8342 0.8028]
Classifier:bag -  F1 Weighted:[0.8358 0.8721 0.845  0.8248 0.856  0.8575 0.8269]
Classifier:ada -  F1 Weighted:[0.82   0.8287 0.8235 0.7927 0.8218 0.8376 0.804 ]
Classifier:dt -  BalancedAccuracy:[0.7317 0.7133 0.7167 0.6778 0.7012 0.7429 0.7129]
Classifier:rf -  BalancedAccuracy:[0.7833 0.8267 0.7867 0.7429 0.7947 0.793  0.7679]
Classifier:bag -  BalancedAccuracy:[0.7983 0.8367 0.8183 0.783  0.823  0.8147 0.7896]
Classifier:ada -  BalancedAccuracy:[0.79   0.7933 0.805  0.7546 0.793  0.798  0.773 ]


**2.2** Describe your classifiers and your results.

YOUR ANSWER HERE

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

YOUR ANSWER HERE

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve a balanced accuracy over 83-84%?

In [17]:
# BEGIN CODE HERE
from sklearn.svm import SVC, NuSVC

cls1 = SVC(C=50, degree=1, gamma="auto", kernel="rbf", probability=True)

cls2 = MLPClassifier(activation="relu", alpha=0.1, hidden_layer_sizes=(10, 10, 10),
                     learning_rate="constant", max_iter=2000, random_state=1000)

cls3 = NuSVC(degree=1, kernel="rbf", nu=0.25, probability=True)

cls4 = RandomForestClassifier(n_estimators=500, criterion="gini", max_depth=10,
                              max_features="auto", min_samples_leaf=0.005,
                              min_samples_split=0.005, n_jobs=-1, random_state=1000)

best_cls = StackingClassifier(estimators=[
                              cls1, cls2, cls3, cls4],  stack_method='auto', n_jobs=-1, passthrough=False)

scores = {'accuracy': make_scorer(
    accuracy_score), 'f1_score': make_scorer(f1_score)}

kfold = KFold(n_splits=10, random_state=RANDOM_STATE, shuffle=True)

results = cross_validate(scls, X, y, cv=kfold, n_jobs=-1, scoring=scoring)

best_fmeasure = results["test_f1_score"]
best_accuracy = results["test_accuracy"]
# END CODE HERE


In [18]:
print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score:{} & Balanced Accuracy:{}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score:[0.87571702 0.88176353 0.89411765 0.8826087  0.87747036 0.859375
 0.89029536 0.89323843 0.84166667 0.88270378] & Balanced Accuracy:[0.8452381  0.85952381 0.87142857 0.87142857 0.85238095 0.82857143
 0.87589499 0.85680191 0.81861575 0.85918854]


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

YOUR ANSWER HERE

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [19]:
# BEGIN CODE HERE
cls = RandomForestClassifier(n_jobs=-1, random_state=RANDOM_STATE)
cls.fit(X,y)
#END CODE HERE
test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both should aim above 85%!